<a href="https://colab.research.google.com/github/edcalderin/DeepLearning_SaturdaysAI/blob/master/3_RecurrentNeuralNets/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embeddings

En la parte teórica hemos visto que para crear *neural networks* para el lenguaje, primero tenemos que transformar el texto en una matriz de vectores. El tipo de vectores que capturan la semántica de las palabras se llaman **word embeddings** y son los que se utilizan en la mayoría de aplicaciones modernas de NLP. Vamos a ver cómo utilizamos embeddings en PyTorch.  

En PyTorch se pueden utilizan los *word embeddings* en inglés más típicos (word2vec, GloVe, FastText...) directamente. Nosotros vamos a trabajar con *embeddings* españoles. Recomiendo bajarse estos vectores https://www.kaggle.com/rtatman/pretrained-word-vectors-for-spanish creados por Cristian Cardellino (atención, es un fichero grande de aprox. 3GB)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading

Loading word embeddings en inglés es muy fácil. PyTorch da acceso a los embeddings más típicos a través de la library **torchtext**. 

In [2]:
# load english vectors
from torchtext import vocab

glove = vocab.GloVe(name='6B', dim=100)

print("Hay {} palabras en el vocabulario".format(len(glove.itos)))

.vector_cache/glove.6B.zip: 862MB [02:40, 5.38MB/s]                           
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-c3745af07af2>", line 4, in <module>
    glove = vocab.GloVe(name='6B', dim=100)
  File "/usr/local/lib/python3.7/dist-packages/torchtext/vocab.py", line 487, in __init__
    super(GloVe, self).__init__(name, url=url, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torchtext/vocab.py", line 326, in __init__
    self.cache(name, cache, url=url, max_vectors=max_vectors)
  File "/usr/local/lib/python3.7/dist-packages/torchtext/vocab.py", line 369, in cache
    zf.extractall(cache)
  File "/usr/lib/python3.7/zipfile.py", line 1636, in extractall
    self._extract_member(zipinfo, path, pwd)
  File "/usr/lib/python3.7/zipfile.py", line 1691, in _extract_member
    shutil.copyfileobj(source, target)
  File "/usr/lib/python3.7/shutil.py", line 79, in copyfileobj
    buf 

KeyboardInterrupt: ignored

Para acceder a los vectores españoles, tenemos que indicar dónde está el fichero.

In [ ]:
# load spanish vectors

es_vectors = vocab.Vectors('SBW-vectors-300-min5.txt', cache='drive/MyDrive/SaturdaysAI')

print("Hay {} palabras en el vocabulario".format(len(es_vectors.itos)))

In [ ]:
# ver dimensiones

es_vectors.vectors.shape

torch.Size([1000653, 300])

# Examinar los embeddings

Podemos examinar los embeddings individualmente y ver qué vector está asociado con qué palabra del vocabulario. 

In [ ]:
# encontrar el índice de una palabra

es_vectors.stoi['perro']

5880

In [ ]:
# examinar vector

es_vectors.vectors[5880]

In [ ]:
# out-of-vocabulary words

es_vectors.stoi['prro']

In [ ]:
# funcion que extrae el word embedding para una palabra

def get_vector(embeddings, word):
    assert word in embeddings.stoi, f'*{word}* no se encuentra en el vocabulario!'
    return embeddings.vectors[embeddings.stoi[word]]

In [ ]:
# examinar vector

get_vector(es_vectors, 'perro').shape

torch.Size([300])

# Contextos similares

Para encontrar palabras similares a una palabra en concreto, primero tenemos que encontrar el vector de esta palabra y luego calcular la distancia entre este vector y los vectores del resto de las palabras. Luego los ordenamos de más cerca a más lejano. 

In [ ]:
# Función para encontrar palabras más similares

import torch

def closest_words(embeddings, vector, n = 10):
    
    distances = [(word, torch.dist(vector, get_vector(embeddings, word)).item())
                 for word in embeddings.itos]
    
    return sorted(distances, key = lambda w: w[1])[:n]

In [ ]:
# Buscar vectores más cercanos

word_vector = get_vector(es_vectors, 'perro')

closest_words(es_vectors, word_vector)

[('perro', 0.0),
 ('perros', 0.7023846507072449),
 ('cachorro', 0.7027554512023926),
 ('gato', 0.7147189378738403),
 ('schnauzer', 0.7251959443092346),
 ('mastín', 0.7283346056938171),
 ('caniche', 0.7307871580123901),
 ('teckel', 0.7311853170394897),
 ('pinscher', 0.7341269850730896),
 ('collie', 0.7447739243507385)]

# Analogía

Otra propiedad de los *word embeddings* es que podemos hacer operaciones como si fueran vectores normales, con resultados interesantes.

In [ ]:
def analogy(embeddings, word1, word2, word3, n=5):
    
    #obtener vectores para cada palaba
    word1_vector = get_vector(embeddings, word1)
    word2_vector = get_vector(embeddings, word2)
    word3_vector = get_vector(embeddings, word3)
    
    #calcularel vector análogo
    analogy_vector = word2_vector - word1_vector + word3_vector
    
    #encontrar palabras más cercanas
    candidate_words = closest_words(embeddings, analogy_vector, n+3)
    
    #filtrar palabras que ya se encuentran en la analogía
    candidate_words = [(word, dist) for (word, dist) in candidate_words 
                       if word not in [word1, word2, word3]][:n]
    
    print(f'{word1} es a {word2} como {word3} es a...')
    
    return candidate_words

In [ ]:
def print_tuples(tuples):
    for w, d in tuples:
        print(f'({d:02.04f}) {w}')

In [ ]:
# buscar analogía

print_tuples(analogy(es_vectors, "rey", "hombre", "reina"))
print_tuples(analogy(es_vectors, 'perro', 'cachorro', 'gato'))